QUESTION 1

Pouvoir d'arrêt collisionnel des protons:

$$\frac{S_{col}}{\rho}=-\left(\frac{d T}{\rho dx}\right)_{col}=NZ\int_0^{T_e^{max}}T'\left(\frac{d\sigma}{dT'}\right)_{col}d T'$$

$$T_e^{max}=\frac{2m_e c^2(\gamma^2 -1)}{1+2\gamma\frac{m_e}{m_p}+\left(\frac{m_e}{m_p}\right)^2}$$

$$-\left(\frac{dT}{dx}\right)_{col}=2\pi r^2_em_ec^2n_e\frac{Z^2}{\beta^2}\left[\ln\left(\frac{2m_ec^2(\gamma^2-1)T_e^{max}}{I^2}\right)-2\beta^2-\delta-2\frac{C}{Z}+2ZL_1+2Z^2L_2\right]$$

$$S_{col}(T)=2\pi r_e^2 m_e c^2 n_e\beta^{-2}\left[\ln\left(\frac{2m_ec^2\beta^2\gamma^2 T_e^{max}}{I^2}\right)-2\beta^2\right]$$

QUESTION 2

Le fait de considérer seulement l'effet de l'interaction entre les électrons et les protons sur le pouvoir est légitime car cet effet dépasse largement les autres effets qui influencent le pouvoir d'arrêt. Les deux autres phénomènes principaux sont l'effet Bremsstrahlung, qui cause des pertes radiatives, et l'interaction entre les protons et les noyaux atomiques. Selon les données du NIST (SOURCE) représentées à la figure 1, on observe que l'effet de l'interaction entre les protons et les noyaux atomiques dans l'os compact est faible comparée à l'interaction entre les protons et les électrons, spécialement pour la plage d'intérêt, c'est-à-dire pour une énergie cinétique de 3 à 250 MeV. Les données du NIST pour l'eau liquide (SOURCE ET FIGURE 2!!???) montrent un comportement similaire. Par exemple, pour une énergie cinétique de 100 MeV, selon le NIST, les pertes causées par l'interaction proton-électron dans l'eau liquide sont de 45,6 MeV tandis que les pertes causées par l'intéraction proton-noyaux atomiques est de 26,0 keV. Les pertes causées par les interactions avec les électrons sont donc environ 1754 fois plus grandes que celles causées par les interactions avec les noyaux atomiques dans cette situation.

<center><img src="PouvoirArretTotalOS.png"/></center>
<center>Figure 1. AJOUTER UN TITRE ET UNE RÉFÉRENCE AU NIST


<center><img src="PouvoirArretTotalEau.png"/></center>
<center>Figure 2. AJOUTER UN TITRE ET UNE RÉFÉRENCE AU NIST

Pour l'effet Bremsstrahlung, on peut déterminer l'ordre de grandeur des pertes radiatives des protons à partir des données sur les pertes radiatives des faisceaux d'électrons dans les différents milieux. Selon les données du NIST (SOURCE), les pertes radiatives maximales pour un faisceau d'électron dans l'eau liquide pour des énergies de 3 à 250 MeV sont de l'ordre de 1 MeV. Pour un faisceau de protons, la force électromagnétique issue de l'intéraction entre les protons et les électrons sera du même ordre de grandeur que pour un faisceau d'électrons, mais, comme la masse du proton est environ 1836 fois (SOURCE) plus grande que celle de l'électron, l'accélération des protons sera alors environ 1836 fois plus petit par la deuxième loi de Newton. Comme l'accélération cause les pertes radiatives, celles-ci seront également 1836 fois plus petites que pour un faisceau d'électrons. Les pertes seront alors de l'ordre du keV et seront négligeables comparées aux pertes dues aux interactions proton-électron, qui sont de l'ordre du MeV. On peut effectuer le même raisonnement pour l'os compact à partir des données du NIST et en arriver à la même conclusion.

QUESTION 3

Comme le pouvoir d'arrêt du milieu représente la quantité d'énergie que le proton perd lorsqu'il se propage dans le milieu, on voit logiquement que le proton pourra se propager plus loin si le pouvoir d'arrêt du milieu est faible. Il se propagera moins loin si le pouvoir d'arrêt est grand. Il est donc cohérent d'intégrer sur l'inverse du pouvoir d'arrêt afin de déterminer la portée du proton.

AJOUTER PLUS D'EXPLICATIONS???

QUESTION 4

On peut réécrire l'équation du pouvoir d'arrêt en fonction de $\gamma$ à l'aide des relations suivantes:
$$U = 2\pi r_e^2m_ec^2n_e$$
$$T = (\gamma-1)m_pc^2$$
$$T_e^{max} = \frac{a(\gamma^2-1)}{b + \delta\gamma}$$
$$\beta^2\gamma^2 = (\gamma^2-1)$$
où $a$, $b$ et $\delta$ sont des constantes qui dépendent des masses de l'électron et du proton. On En remplaçant dans dans l'équation du pouvoir d'arrêt, on a
$$S_{col} = U\frac{\gamma^2}{(\gamma^2-1)}\Bigg[\ln\bigg(\frac{2m_ec^2a(\gamma^2-1)^2}{I^2(b + \delta)}\bigg)\Bigg] - 2U$$
En posant le changement de variable d'intégration $T = (\gamma-1)m_pc^2$, on obtient une intégrale de la forme suivante pour la portée:
$$R_{CSDA} = \rho m_pc^2\int \frac{\text{d}\gamma}{U\frac{\gamma^2}{(\gamma^2-1)}\Bigg[\ln\bigg(\frac{2m_ec^2a(\gamma^2-1)^2}{I^2(b + \delta)}\bigg)\Bigg] - 2U}$$
Une intégrale de cette forme ne possède pas de solution analytique connue, donc on doit la résoudre numériquement. On peut vérifier qu'elle n'a pas de solution analytique avec un logiciel de calcul tel que WolframAlpha.

QUESTION 7

On utilise l'expression pour le pouvoir d'arrêt en fonction de $\gamma$ déterminé à la question 4:
$$S_{col} = U\frac{\gamma^2}{(\gamma^2-1)}\Bigg[\ln\bigg(\frac{2m_ec^2a(\gamma^2-1)^2}{I^2(b + \delta)}\bigg)\Bigg] - 2U$$
On désire utiliser la formule pour la dérivée d'une multiplication et la dérivation en chaîne. On calcule les dérivées nécessaires:
$$\frac{\text{d}}{\text{d}\gamma} \frac{\gamma^2}{\gamma^2-1} = \Bigg(\frac{2\gamma}{\gamma^2-1} - \frac{2\gamma^3}{(\gamma^2-1)^2}\Bigg) = \frac{-2\gamma}{(\gamma^2-1)^2}$$
$$\frac{\text{d}}{\text{d}\gamma} \frac{2m_ec^2a(\gamma^2-1)^2}{I^2(b + \delta)}\bigg) = \frac{I^2(b+\delta\gamma)}{2m_ec^2a(\gamma^2-1)} \Bigg(\frac{8m_ec^2a\gamma(\gamma^2-1)}{I^2(b+\delta\gamma)} - \frac{2\delta}{\delta(b+\delta \gamma)}\Bigg)$$
$$\frac{\text{d}}{\text{d}\gamma} \frac{2m_ec^2a(\gamma^2-1)^2}{I^2(b + \delta)}\bigg) = \Bigg(\frac{4\gamma}{\gamma^2-1} - \frac{\delta}{\delta(b+\delta \gamma)}\Bigg)$$
$$\frac{\text{d}\gamma}{\text{d}T} = \frac{1}{m_pc^2}$$
On peut alors déterminer la dérivée première du pouvoir d'arrêt:
$$\frac{\text{d}S_{col}}{\text{d}T} = \frac{\text{d}S_{col}}{\text{d}\gamma}\frac{\text{d}\gamma}{\text{d}T}$$
$$\frac{\text{d}S_{col}}{\text{d}T} = \frac{U}{m_pc^2}\Bigg[\frac{-2\gamma}{(\gamma^2-1)^2}\ln\bigg(\frac{2m_ec^2a(\gamma^2-1)^2}{I^2(b + \delta)}\bigg) + \frac{\gamma^2}{(\gamma^2-1)}\Bigg(\frac{4\gamma}{\gamma^2-1} - \frac{\delta}{b+\delta \gamma}\Bigg)\Bigg]$$
$$\frac{\text{d}S_{col}}{\text{d}T} = \frac{U}{m_pc^2}\frac{\gamma}{(\gamma^2-1)^2}\Bigg[4\gamma^2 - 2\ln\bigg(\frac{2m_ec^2a(\gamma^2-1)^2}{I^2(b + \delta)}\bigg) - \frac{\delta\gamma(\gamma^2-1)}{b+\delta\gamma}\Bigg]$$

QUESTION 8

